In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import roots_legendre
from scipy.integrate import fixed_quad, quad, romberg


In [ ]:

def initialize_lattice(L):
    """Generate a random LxL Ising lattice with spins of -1 or 1."""
    return np.random.choice([-1, 1], size=(L, L))

def compute_energy(lattice, J=1, B=0):
    """Calculate the energy of the Ising lattice."""
    L = lattice.shape[0]
    energy = 0
    for i in range(L):
        for j in range(L):
            S = lattice[i, j]
            neighbors = lattice[(i+1)%L, j] + lattice[i, (j+1)%L] + lattice[(i-1)%L, j] + lattice[i, (j-1)%L]
            energy += -J * S * neighbors - B * S
    return energy / 2  # Each pair counted twice

def metropolis_step(lattice, beta, J=1, B=0):
    """Perform a single Metropolis-Hastings update on the lattice."""
    L = lattice.shape[0]
    for _ in range(L**2):
        i, j = np.random.randint(0, L, size=2)
        S = lattice[i, j]
        neighbors = lattice[(i+1)%L, j] + lattice[i, (j+1)%L] + lattice[(i-1)%L, j] + lattice[i, (j-1)%L]
        dE = 2 * J * S * neighbors + 2 * B * S
        if dE < 0 or np.random.rand() < np.exp(-beta * dE):
            lattice[i, j] *= -1

def monte_carlo_simulation(L, T, steps=10000):
    """Run Monte Carlo simulation for the Ising model."""
    beta = 1 / T
    lattice = initialize_lattice(L)
    magnetization = []
    for _ in range(steps):
        metropolis_step(lattice, beta)
        magnetization.append(np.abs(np.sum(lattice)) / (L**2))
    return lattice, magnetization

def plot_magnetization_vs_temperature(L, T_values, steps=10000):
    """Plot the magnetization as a function of temperature."""
    magnetizations = []
    for T in T_values:
        _, mag = monte_carlo_simulation(L, T, steps)
        magnetizations.append(np.mean(mag[-1000:]))
    plt.figure()
    plt.plot(T_values, magnetizations, marker='o')
    plt.xlabel("Temperature (T)")
    plt.ylabel("Magnetization")
    plt.title(f"Magnetization vs Temperature for L={L}")
    plt.axvline(2 / np.log(1 + np.sqrt(2)), color='r', linestyle='--', label='Critical Temperature')
    plt.legend()
    plt.show()


In [ ]:

# Quadrature Class
class Quadrature:
    def midpoint_rule(self, f, a, b):
        return (b - a) * f((a + b) / 2)
    
    def trapezoidal_rule(self, f, a, b):
        return (b - a) / 2 * (f(a) + f(b))
    
    def simpsons_rule(self, f, a, b):
        return (b - a) / 6 * (f(a) + 4 * f((a + b) / 2) + f(b))

class GaussQuadrature(Quadrature):
    def __init__(self, order):
        self.order = order
        self.points, self.weights = roots_legendre(order)
    
    def integrate(self, f, a, b):
        transformed_points = 0.5 * (b - a) * self.points + 0.5 * (a + b)
        return 0.5 * (b - a) * np.sum(self.weights * f(transformed_points))

# Harmonic Oscillator Period Calculation
def potential(x):
    return x**4

def period_integrand(x, a):
    return 1 / np.sqrt(potential(a) - potential(x))

def calculate_period(a, N):
    return np.sqrt(8) * fixed_quad(period_integrand, 0, a, args=(a,), n=N)[0]

def find_N_for_error_tolerance(a=2, tol=1e-4):
    N = 2
    while True:
        T_N = calculate_period(a, N)
        T_2N = calculate_period(a, 2*N)
        if np.abs(T_N - T_2N) < tol:
            return N
        N *= 2

def plot_period_vs_amplitude():
    amplitudes = np.linspace(0.1, 2, 20)
    periods = [calculate_period(a, 50) for a in amplitudes]
    plt.figure()
    plt.plot(amplitudes, periods, marker='o')
    plt.xlabel("Amplitude (a)")
    plt.ylabel("Period (T)")
    plt.title("Harmonic Oscillator Period vs Amplitude")
    plt.show()

# Parameters
L = 10
T_values = np.linspace(1, 4, 10)
plot_magnetization_vs_temperature(L, T_values)
plot_period_vs_amplitude()
